In [ ]:
!pip install transformers

In [ ]:
!pip install sacremoses

In [1]:
import pandas as pd
from transformers import FSMTForConditionalGeneration, FSMTTokenizer


In [2]:
modelNameRuEn = "facebook/wmt19-ru-en"

tokenizerRuEn = FSMTTokenizer.from_pretrained(modelNameRuEn)
modelRuEn = FSMTForConditionalGeneration.from_pretrained(modelNameRuEn)

In [3]:
def translateRuEn(inputRu):
  input = inputRu
  input_ids = tokenizerRuEn.encode(input, return_tensors="pt")
  outputs = modelRuEn.generate(input_ids)
  decodedEn = tokenizerRuEn.decode(outputs[0], skip_special_tokens=True)
  return decodedEn

In [4]:
firstSet = pd.read_json('./firstSet.json')
firstSet

,question,passage,label,idx
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...,1.0,0
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,...",1.0,1
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...,1.0,2
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010...",1.0,3
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...,0.0,4
...,...,...,...,...
2549,Была ли у джихангира наложница?,Хюрре́м Хасеки́-султа́н ; настоящее имя неизве...,NaN,800
2550,Правда ли у животных нет групп крови?,Гру́ппа кро́ви — описание индивидуальных антиг...,NaN,801
2551,Все ли погибли на титанике?,"Элизабет Глэдис Миллвина Дин — англичанка, ко...",NaN,802
2552,Есть ли в хлебе белок?,Эта статья о группе белков. О пищевом продукте...,NaN,803


In [5]:
newSet = []
for i in range(len(firstSet)):
    if i == 0 or firstSet.question[i] != firstSet.question[i-1]:
        newSetElem = dict()
        newSetElem['question'] = firstSet.question[i]
        newSetElem['passage'] = firstSet.passage[i]
        newSet.append(newSetElem)
    else:
        newSetElem['passage'] = newSetElem['passage'] + firstSet.passage[i]

In [6]:
pd.DataFrame(newSet).to_json('./dataset.json')

In [7]:
dataJSON = pd.read_json('./dataset.json')
dataJSON

,question,passage
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...
1,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...
2,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010..."
3,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...
4,Может ли автомобиль ездить на газу?,Автомобиль на природном газе — один из видов а...
...,...,...
2199,Была ли у джихангира наложница?,Хюрре́м Хасеки́-султа́н ; настоящее имя неизве...
2200,Правда ли у животных нет групп крови?,Гру́ппа кро́ви — описание индивидуальных антиг...
2201,Все ли погибли на титанике?,"Элизабет Глэдис Миллвина Дин — англичанка, ко..."
2202,Есть ли в хлебе белок?,Эта статья о группе белков. О пищевом продукте...


In [ ]:

dataJSON.questionEN = []
for i in range(len(dataJSON.question)):
   if i % 50 == 0:
      ptint(i)
   dataJSON.questionEN.append(translateRuEn(dataJSON.question[i]))

In [ ]:
pd.DataFrame(dataJSON.questionEN).to_csv('./qEn.csv',header = None)

In [ ]:
questionENShort = []
for i in range(100):
    questionENShort.append(dataJSON.questionEN[i])
len(questionENShort)

100

In [ ]:
pd.DataFrame(questionENShort).to_csv('./qEnShort.csv',header = None)